<a href="https://colab.research.google.com/github/salimnh/chronic-lymphocytic-leukemia/blob/master/LLC_Detection_Quinten_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

import numpy as np
import pandas as pd


import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px

import seaborn as sns



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [0]:
path='/content/gdrive/My Drive/raw_data.csv'

In [0]:
df=pd.read_csv(path,sep=";")
a=df.groupby(['patient_name','patient_state']).agg({'cell_name':'count'}).reset_index()
a.patient_state.value_counts()

sain      19
malade    14
Name: patient_state, dtype: int64

# Util Function

In [0]:
def conditions_cell_type_B_LLC(df):
    if df['cell_type']== "B" and df['patient_state'] == "sain":
      return "B"
    elif (df['cell_type']== "B") and (df['patient_state'] == "malade"):
      return "B_LLC"
    else:
      return "TNK"




In [0]:
def get_patient_state_box_plt(df):
  df1=df[['patient_name', 'cell_name','cell_type','patient_state','spectre']]
  df1['LB']= np.where(df['cell_type'].str.contains("B"),1,0)
  df3=df1.groupby(["patient_name","patient_state"]).agg({'LB':'mean','cell_name':'count'}).reset_index()
  fig = px.box(df3, x="patient_state", y="LB",points="all")
  fig.show()

  

In [0]:
def get_spectra(df=df,group="cell_type_new",agg="mean"):
  if agg=="mean":
    test=df.groupby([group],as_index=False)[df.filter(regex='lambda_.*').columns].mean()
  elif agg=="var":
    test=df.groupby([group],as_index=False)[df.filter(regex='lambda_.*').columns].var()

  test['sample_index'] = range(1, len(test) + 1)
  test

  test1 = (test.set_index([group,"sample_index"])
          .stack()
          .reset_index(name='Value')
          .rename(columns={'level_2':'lambda'}))
  test1['lambda'] = test1['lambda'].str.extract('(\d+)', expand=False)
  #df1=df1.iloc[45954:49950,:]

  fig = px.line(test1, x="lambda", y="Value", color=group,
                line_group="sample_index", hover_name="sample_index")
  return fig.show()

# EDA

### #cellType


In [0]:
df['cell_type_new']=df.apply(conditions_cell_type_B_LLC, axis=1)
df.cell_type_new.value_counts()

TNK      4289
B_LLC    2271
B         637
Name: cell_type_new, dtype: int64

In [0]:
df.groupby(['patient_name','cell_type']).agg({'cell_name':'count'}).reset_index()


,patient_name,cell_type,cell_name
0,P1_ULC100,B,4
1,P1_ULC100,TNK,170
2,P1_ULC101,B,15
3,P1_ULC101,TNK,164
4,P1_ULC103,TNK,88
...,...,...,...
56,P2_ULC93,TNK,7
57,P2_ULC96,B,192
58,P2_ULC96,TNK,3
59,P2_ULC99,B,61


### Lynphocytes proportion: B vs TNK 

In [0]:
get_patient_state_box_plt(df)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Cell Type mean

In [0]:
get_spectra(df,group="cell_type_new",agg="mean")

### Variance per cell type


In [0]:
get_spectra(df,group="cell_type_new",agg="var")